# Chatbot Educativo con Naive Bayes y Similitud Semántica
Este notebook desarrolla un chatbot que clasifica las preguntas de los estudiantes y selecciona la mejor respuesta posible con base en análisis semántico.

## ¿Qué es Naive Bayes?
Naive Bayes es un clasificador probabilístico basado en el teorema de Bayes. Asume que las características (palabras, en este caso) son independientes entre sí. Es eficaz para tareas de clasificación de texto, como spam, sentimientos o, en este caso, la clasificación de intenciones.

## ¿Qué es TF-IDF?
**TF-IDF** (Term Frequency - Inverse Document Frequency) es una técnica que transforma texto en valores numéricos que reflejan la importancia de las palabras en los documentos. Se utiliza para vectorizar texto antes de aplicar modelos de aprendizaje automático.

## ¿Qué es la similitud semántica?
La **similitud semántica** mide cuán similares son dos textos en términos de significado. En este caso usamos vectores de SpaCy para representar frases, y comparamos la consulta del usuario con las posibles respuestas para elegir la más adecuada.

## 📐 Fundamentos Matemáticos

### 🔢 Teorema de Bayes aplicado a texto
El clasificador Naive Bayes predice la clase \( C_k \) dada una observación \( x \) usando la siguiente fórmula:

\[ P(C_k \mid x) = \frac{P(x \mid C_k) P(C_k)}{P(x)} \]

En clasificación de texto, \( x \) son las palabras del documento. Bajo la suposición de independencia ingenua entre palabras, se convierte en:

\[ P(C_k \mid x_1, ..., x_n) \propto P(C_k) \prod_{i=1}^n P(x_i \mid C_k) \]

### 🧮 Fórmula de TF-IDF
**TF (Frecuencia de Término):** cuántas veces aparece un término en un documento.

\[ TF(t, d) = \frac{f_{t,d}}{\sum_k f_{k,d}} \]

**IDF (Frecuencia Inversa de Documentos):** mide cuán común o rara es una palabra en todos los documentos.

\[ IDF(t, D) = \log \left( \frac{N}{1 + n_t} \right) \]

Donde:
- \( f_{t,d} \) es la frecuencia del término \( t \) en el documento \( d \)
- \( N \) es el número total de documentos
- \( n_t \) es el número de documentos que contienen el término \( t \)

**TF-IDF final:**
\[ TFIDF(t, d, D) = TF(t, d) \cdot IDF(t, D) \]

### 🔗 Similitud de coseno
Se usa para medir qué tan similares son dos vectores (frase del usuario y respuesta).
Se calcula así:

\[ \text{coseno}(A, B) = \frac{A \cdot B}{\|A\| \|B\|} = \frac{\sum_{i=1}^n A_i B_i}{\sqrt{\sum_{i=1}^n A_i^2} \sqrt{\sum_{i=1}^n B_i^2}} \]

El valor resultante está entre 0 (completamente diferente) y 1 (idéntico).

# Chatbot Educativo con Naive Bayes y TF-IDF

### Entrenamiento del clasificador con TF-IDF y Naive Bayes
Creamos un pipeline que convierte las frases limpias en vectores usando TF-IDF, y luego las clasifica con un modelo de Naive Bayes entrenado sobre las etiquetas del dataset.

**Chatbot Educativo con Naive Bayes y TF-IDF**
Este cuaderno entrena un modelo Naive Bayes para responder preguntas frecuentes sobre temas académicos como matrículas, horarios, calificaciones, etc.


In [15]:
#importar libreria
import pandas as pd
import spacy
import unicodedata
import re
from sklearn.metrics.pairwise import cosine_similarity
import random
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

nlp = spacy.load("es_core_news_md")

In [16]:
## 📥 Cargar el Dataset
df = pd.read_csv(r"C:\Users\darly\Downloads\dataEducacion_final_393_agregado.csv")

df.head()

,consulta,categoria,respuesta,respuesta_compuesta,respuestas_compuestas
0,¿Cómo me inscribo al próximo semestre?,matriculas,"Para inscribirte al próximo semestre, debes in...","Para me inscribo al próximo semestre, Para ins...","Para inscribirte inscribo al próximo semestre,..."
1,¿Cuál es la fecha límite para inscribirse?,matriculas,La fecha límite para completar tu inscripción ...,"La la fecha límite para inscribirse es, La fec...","La la fecha límite para inscribirse es, La fec..."
2,¿Qué documentos necesito para la matrícula?,matriculas,Para realizar tu matrícula necesitas: identifi...,Lo que necesitas saber sobre documentos necesi...,Lo que necesitas saber sobre documentos necesi...
3,¿Cuánto cuesta la inscripción?,matriculas,El costo de inscripción para el semestre actua...,El valor relacionado con cuesta la inscripción...,El valor relacionado con cuesta la inscripción...
4,¿Dónde puedo pagar mi matrícula?,matriculas,Puedes pagar tu matrícula en cualquier sucursa...,"Puedes hacerlo en el lugar donde, Puedes pagar...","Puedes realizarlo en el lugar donde, Puedes pa..."


In [17]:
#limpieza de los datos #Incluye:
#- Minúsculas
#- Eliminación de tildes
#- Lematización
#- Conserva palabras importantes como "dónde", "cuándo", "cómo", "qué", etc.


# Palabras que queremos conservar (normalizadas)
custom_stopwords_to_keep = {"no", "si", "donde", "cuando", "como", "que", "cual", "cuanto"}

def normalize_text(text):
    # Paso 1: convertir a minúsculas y eliminar tildes
    text = str(text).lower()
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    text = re.sub(r"[^a-zA-Züñ¿?¡! ]", "", text)  # conservamos solo letras simples

    # Paso 2: procesar con Spacy
    doc = nlp(text)
    
    # Paso 3: eliminar stopwords solo si no están en custom_stopwords_to_keep
    tokens = [
        token.lemma_ for token in doc
        if not (token.is_stop and token.lemma_.lower() not in custom_stopwords_to_keep)
    ]
    
    return " ".join(tokens)


In [18]:
# Aplicar limpieza
df["consulta_limpia"] = df["consulta"].apply(normalize_text)

In [19]:
#Vamos a crear un modelo de clasificación de intención por categoría usando n-gramas de 1 a 3.

X = df["consulta_limpia"]
y = df["categoria"]

In [20]:
#partir set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
#crear estructura de entranamiento y medir la clasificacion

model = make_pipeline(
    TfidfVectorizer(
        ngram_range=(1, 2),       # Usa un contexto de hasta 3 palabras
        min_df=1                # Aparece en al menos 2 consulta
                   ),
    MultinomialNB()
)


In [22]:
#entrenar modelo
model.fit(X_train, y_train)
print("Precisión del modelo:", model.score(X_test, y_test))
preds = model.predict(X_test)
print(classification_report(y_test, preds))


Precisión del modelo: 0.8734177215189873
                precision    recall  f1-score   support

calificaciones       0.87      0.93      0.90        14
      horarios       0.82      1.00      0.90        14
 instalaciones       1.00      0.92      0.96        12
    matriculas       0.83      0.91      0.87        11
    requisitos       0.83      0.71      0.77        14
      tramites       0.92      0.79      0.85        14

      accuracy                           0.87        79
     macro avg       0.88      0.88      0.87        79
  weighted avg       0.88      0.87      0.87        79



In [30]:
## 💬 Motor de Chatbot
#Definimos saludos y despedidas y usamos clasificación + similitud para elegir la mejor respuesta en cada categoría.

saludos = ["hola", "buenos dias", "buenas tardes", "buenas noches", "hola que tal", "como estas", "ey"]
respuestas_saludo = ["¡Hola! ¿En qué puedo ayudarte?", "¡Buenos días! ¿Qué necesitas?", "Hola, dime tu duda."]

despedidas = ["gracias", "hasta luego", "adios", "nos vemos", "bye", "chao"]
respuestas_despedida = ["¡Hasta pronto!", "Gracias por tu consulta. ¡Éxitos!", "Nos vemos."]

def responder(pregunta):
    pregunta_limpia = normalize_text(pregunta)
    print("🔍 Consulta limpia →", pregunta_limpia)
    
    if any(saludo in pregunta.lower() for saludo in saludos):
        return random.choice(respuestas_saludo)
    
    if any(despedida in pregunta.lower() for despedida in despedidas):
        return random.choice(respuestas_despedida)
    
    # 1. Clasificar la intención (etiqueta)
    etiqueta = model.predict([pregunta_limpia])[0]
    print("📌 Categoría detectada →", etiqueta)
    
    # 2. Filtrar respuestas de esa categoría
    respuestas_categoria = df[df["categoria"] == etiqueta].copy()

    # 3. Normalizar las respuestas para compararlas
    respuestas_categoria["respuesta_limpia"] = respuestas_categoria["respuesta_compuesta"].apply(normalize_text)

    # 4. Vectorizar respuestas limpias
    vectorizer = model.named_steps["tfidfvectorizer"]
    respuestas_categoria["respuesta_vec"] = respuestas_categoria["respuesta_limpia"].apply(lambda x: vectorizer.transform([x]))
    
    pregunta_vec = vectorizer.transform([pregunta_limpia])
    
    # 5. Calcular similitud contra respuestas
    respuestas_categoria["similitud"] = respuestas_categoria["respuesta_vec"].apply(lambda x: cosine_similarity(x, pregunta_vec)[0][0])
    
    # 6. Mostrar top 5 respuestas más similares
    print("\n🎯 Top 5 respuestas más similares:")
    top5 = respuestas_categoria[["respuesta_compuesta", "similitud"]].sort_values(by="similitud", ascending=False).head(5)
    print(top5.to_string(index=False))

    # 7. Devolver la mejor respuesta
    mejor_idx = respuestas_categoria["similitud"].idxmax()
  
    return df.loc[mejor_idx, "respuesta"]


In [31]:

#prueba

print(responder("hola"))
print()
print(responder("¿Cómo me inscribo a clases?"))
print(responder("¿Dónde veo mis notas?"))
print(responder("Gracias por la información"))


🔍 Consulta limpia → hola
Hola, dime tu duda.

🔍 Consulta limpia → ¿ como inscribo clase ?
📌 Categoría detectada → horarios

🎯 Top 5 respuestas más similares:
                                                                                                                                                                                                                                                                                                      respuesta_compuesta  similitud
                                                   Con respecto a tu pregunta sobre a qué hora comienzan las clases,, Las clases matutinas comienzan a las 7:00 AM, el turno vespertino inicia a las 2:00 PM y las clases nocturnas a partir de las 6:00 PM. Te recomendamos llegar 15 minutos antes para ubicar tu aula.   0.237527
Con respecto a tu pregunta sobre puedo tomar clases en turno vespertino y matutino,, Sí, puedes combinar clases de diferentes turnos siempre que no existan traslapes de horario y cumplas con l

In [25]:
print(responder("cuales son las instalaciones de la universidad"))

🔍 Consulta limpia → cual instalación universidad
📌 Categoría detectada → instalaciones

🎯 Top 5 respuestas más similares:
                                                                                                                                                                                                                                                    respuesta_compuesta  similitud
Lo que necesitas saber sobre servicios tecnológicos ofrece la universidad a los estudiantes es, La universidad ofrece acceso a Wi-Fi en todo el campus, salas de cómputo, plataformas virtuales, correo institucional y licencias educativas de software especializado.   0.194219
                                                                                                                              Para accedo a las instalaciones deportivas, Con tu credencial institucional puedes ingresar en los horarios autorizados para estudiantes.   0.137562
                                                     